In [15]:
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re, collections
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import cohen_kappa_score
import enchant 
import pandas as pd
import spacy
import re
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
import urllib.request

In [4]:
dataframe = pd.read_csv('training_set_rel3.tsv', encoding = 'latin-1',sep='\t')
dataframe = dataframe[['essay_id','essay_set','essay','domain1_score']]
dataframe.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [5]:
dataframe = dataframe.iloc[:10]
dataframe.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [6]:
dataframe.set_index('essay_id',inplace=True, drop=True)

In [10]:
def get_number_of_characters(df):
    chars = []
    for essay in df['essay']:
        chars.append(len(essay))
        
    return df.assign(num_of_chars = chars) 

dataframe = get_number_of_characters(dataframe) 
dataframe.head()

,essay_set,essay,domain1_score,num_of_chars
essay_id,,,,
1,1,"Dear local newspaper, I think effects computer...",8,1875
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,2288
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,1541
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,3165
5,1,"Dear @LOCATION1, I know having computers has a...",8,2569


In [17]:
tokenizer = RegexpTokenizer(r'\w+')
def get_number_of_words(regExTokenizer, df):
    length = []
    for essay in df['essay']:
        length.append(len(regExTokenizer.tokenize(essay)))
        
    return df.assign(num_of_words = length) 

dataframe = get_number_of_words(tokenizer, dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_chars,length,num_of_words
essay_id,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,1875,350,350
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,2288,423,423
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,1541,283,283
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,3165,530,530
5,1,"Dear @LOCATION1, I know having computers has a...",8,2569,473,473


In [19]:
def get_number_of_long_words(regExTokenizer,df):
    length = []
    for essay in df['essay']:
        long_words = 0
        for word in regExTokenizer.tokenize(essay):
            if len(word)>8:
                long_words+=1
        length.append(long_words)
    return df.assign(num_of_long_words = length)
dataframe = get_number_of_long_words(tokenizer, dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_chars,num_of_words,num_of_long_words
essay_id,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,1875,350,13
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,2288,423,29
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,1541,283,19
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,3165,530,55
5,1,"Dear @LOCATION1, I know having computers has a...",8,2569,473,36


In [20]:
def get_number_of_short_words(regExTokenizer,df):
    length = []
    for essay in df['essay']:
        short_words = 0
        for word in regExTokenizer.tokenize(essay):
            if len(word)<=3:
                short_words+=1
        length.append(short_words)
    return df.assign(num_of_short_words = length)
dataframe = get_number_of_short_words(tokenizer, dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_chars,num_of_words,num_of_long_words,num_of_short_words
essay_id,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,1875,350,13,151
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,2288,423,29,175
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,1541,283,19,126
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,3165,530,55,189
5,1,"Dear @LOCATION1, I know having computers has a...",8,2569,473,36,205


In [21]:
def get_most_frequent_word_length(regExTokenizer,df):
    length = []
    for essay in df['essay']:
        len_dict = {}
        for word in regExTokenizer.tokenize(essay):
            if len(word) in len_dict:
                len_dict[len(word)] = len_dict[len(word)]+1
            else:
                len_dict[len(word)] = 1
        maxwords = max(len_dict,key=len_dict.get)
        length.append(maxwords)
    return df.assign(most_frequent_word_length = length)
dataframe = get_most_frequent_word_length(tokenizer, dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_chars,num_of_words,num_of_long_words,num_of_short_words,most_frequent_word_length
essay_id,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,1875,350,13,151,2
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,2288,423,29,175,4
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,1541,283,19,126,3
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,3165,530,55,189,4
5,1,"Dear @LOCATION1, I know having computers has a...",8,2569,473,36,205,4


In [27]:
def get_average_word_length(df):
    length = []
    for i in range(0,df.shape[0]):
        row = df.iloc[i]
        average_word_length = (row['num_of_chars']-row['num_of_words'])/row['num_of_words']
        length.append(average_word_length)
    return df.assign(average_word_length = length)
dataframe = get_average_word_length(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_chars,num_of_words,num_of_long_words,num_of_short_words,most_frequent_word_length,average_word_length
essay_id,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,1875,350,13,151,2,4.357143
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,2288,423,29,175,4,4.408983
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,1541,283,19,126,3,4.445230
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,3165,530,55,189,4,4.971698
5,1,"Dear @LOCATION1, I know having computers has a...",8,2569,473,36,205,4,4.431290


In [30]:
nlp = spacy.load('en_core_web_md')
def get_number_of_sentences(df):
    _byRow_sents = []
    for essay in df['essay']:
        sents = []
        parsed_essay = nlp(essay)
        for num, sentence in enumerate(parsed_essay.sents):
            sents.append(sentence)
        _byRow_sents.append(len(sents))   
    return df.assign(num_of_sentences = _byRow_sents)

dataframe = get_number_of_sentences(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_chars,num_of_words,num_of_long_words,num_of_short_words,most_frequent_word_length,average_word_length,num_of_sentences
essay_id,,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,1875,350,13,151,2,4.357143,16
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,2288,423,29,175,4,4.408983,21
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,1541,283,19,126,3,4.445230,17
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,3165,530,55,189,4,4.971698,29
5,1,"Dear @LOCATION1, I know having computers has a...",8,2569,473,36,205,4,4.431290,30


In [36]:
def get_number_of_sentence_attributes(regExTokenizer,df):
    #number of long sentences
    #number of short sentences
    #average sentence length
    #most frequent sentence length
    num_of_long_sents = []
    num_of_short_sents = []
    average_sents_length = []
    most_frequent_sents_length = []
    for essay in df['essay']:
        long_sents = 0
        short_sents = 0
        sent_dict = {}
        num_of_sents = 0
        num_of_words = 0
        parsed_essay = nlp(essay)
        for num, sentence in enumerate(parsed_essay.sents):
            tokenized = len(regExTokenizer.tokenize(sentence.text))
            if tokenized > 25:
                long_sents+=1
            if tokenized < 10:
                short_sents+=1
            num_of_words+=tokenized
            num_of_sents+=1
            if tokenized in sent_dict:
                sent_dict[tokenized] = sent_dict[tokenized]+1
            else:
                sent_dict[tokenized] = 1
        num_of_long_sents.append(long_sents)
        num_of_short_sents.append(short_sents)
        average_sents_length.append(num_of_words/num_of_sents)
        maxwords = max(sent_dict,key=sent_dict.get)
        most_frequent_sents_length.append(maxwords)
    df = df.assign(num_of_long_sents = num_of_long_sents)
    df = df.assign(num_of_short_sents = num_of_short_sents)
    df = df.assign(average_sents_length = average_sents_length)
    df = df.assign(most_frequent_sents_length = most_frequent_sents_length)
    return df

dataframe = get_number_of_sentence_attributes(tokenizer,dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_chars,num_of_words,num_of_long_words,num_of_short_words,most_frequent_word_length,average_word_length,num_of_sentences,num_of_long_sents,num_of_short_sents,average_sents_length,most_frequent_sents_length
essay_id,,,,,,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,1875,350,13,151,2,4.357143,16,6,3,21.875000,4
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,2288,423,29,175,4,4.408983,21,6,4,20.142857,15
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,1541,283,19,126,3,4.445230,17,4,6,16.647059,4
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,3165,530,55,189,4,4.971698,29,6,4,18.275862,13
5,1,"Dear @LOCATION1, I know having computers has a...",8,2569,473,36,205,4,4.431290,30,3,3,15.766667,12


In [37]:
def get_number_of_different_words(regExTokenizer,df):
    length = []
    for essay in df['essay']:
        word_dict = {}
        for word in regExTokenizer.tokenize(essay):
            if word in word_dict:
                word_dict[word] = word_dict[word]+1
            else:
                word_dict[word] = 1
        length.append(len(word_dict))
    return df.assign(num_of_different_words = length)
dataframe = get_number_of_different_words(tokenizer, dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_chars,num_of_words,num_of_long_words,num_of_short_words,most_frequent_word_length,average_word_length,num_of_sentences,num_of_long_sents,num_of_short_sents,average_sents_length,most_frequent_sents_length,num_of_different_words
essay_id,,,,,,,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,1875,350,13,151,2,4.357143,16,6,3,21.875000,4,173
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,2288,423,29,175,4,4.408983,21,6,4,20.142857,15,205
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,1541,283,19,126,3,4.445230,17,4,6,16.647059,4,160
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,3165,530,55,189,4,4.971698,29,6,4,18.275862,13,260
5,1,"Dear @LOCATION1, I know having computers has a...",8,2569,473,36,205,4,4.431290,30,3,3,15.766667,12,210


In [41]:
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def get_number_of_stopwords(df):
    stop_words = set(stopwords.words('english'))
    length = []
    for essay in df['essay']:
        word_tokens = word_tokenize(essay)
        filtered_sentence = [w for w in word_tokens if w in stop_words]
        length.append(len(filtered_sentence))
    return df.assign(num_of_stopwords = length)
dataframe = get_number_of_stopwords(dataframe)
dataframe.head()
    

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aman\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


,essay_set,essay,domain1_score,num_of_chars,num_of_words,num_of_long_words,num_of_short_words,most_frequent_word_length,average_word_length,num_of_sentences,num_of_long_sents,num_of_short_sents,average_sents_length,most_frequent_sents_length,num_of_different_words,num_of_stopwords
essay_id,,,,,,,,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,1875,350,13,151,2,4.357143,16,6,3,21.875000,4,173,165
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,2288,423,29,175,4,4.408983,21,6,4,20.142857,15,205,181
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,1541,283,19,126,3,4.445230,17,4,6,16.647059,4,160,132
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,3165,530,55,189,4,4.971698,29,6,4,18.275862,13,260,208
5,1,"Dear @LOCATION1, I know having computers has a...",8,2569,473,36,205,4,4.431290,30,3,3,15.766667,12,210,216


In [44]:
from readcalc import readcalc
import textstat
def get_readability_measures(df):
    gunning_fox_index = []
    flesch_reading_ease = []
    flesch_kincaid_grade_level = []
    dale_schall_index = []
    automated_readability_index = []
    SMOG = []
    LIX = []
    for essay in df['essay']:
        flesch_kincaid_grade_level.append(textstat.flesch_kincaid_grade(essay))
        gunning_fox_index.append(textstat.gunning_fog(essay))
        flesch_reading_ease.append(textstat.flesch_reading_ease(essay))
        dale_schall_index.append(textstat.dale_chall_readability_score(essay))
        automated_readability_index.append(textstat.automated_readability_index(essay))
        SMOG.append(textstat.smog_index(essay))
        calc = readcalc.ReadCalc(essay)
        LIX.append(calc.get_lix_index())
    df = df.assign(gunning_fox_index = gunning_fox_index)
    df = df.assign(flesch_reading_ease = flesch_reading_ease)
    df = df.assign(flesch_kincaid_grade_level = flesch_kincaid_grade_level)
    df = df.assign(dale_schall_index = dale_schall_index)
    df = df.assign(automated_readability_index = automated_readability_index)
    df = df.assign(SMOG = SMOG)
    df = df.assign(LIX = LIX)
    return df
dataframe = get_readability_measures(dataframe)
dataframe.head()
        

TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 1875
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 2288
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 1541
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 3165
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 2569
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 1276
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 2808
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 2724
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 2402
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 2632


,essay_set,essay,domain1_score,num_of_chars,num_of_words,num_of_long_words,num_of_short_words,most_frequent_word_length,average_word_length,num_of_sentences,...,most_frequent_sents_length,num_of_different_words,num_of_stopwords,gunning_fox_index,flesch_reading_ease,flesch_kincaid_grade_level,dale_schall_index,automated_readability_index,SMOG,LIX
essay_id,,,,,,,,,,,,,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,1875,350,13,151,2,4.357143,16,...,4,173,165,10.31,74.02,8.5,6.06,11.3,10.2,38.638666
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,2288,423,29,175,4,4.408983,21,...,15,205,181,10.21,67.08,9.1,6.60,10.1,11.6,40.294313
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,1541,283,19,126,3,4.445230,17,...,4,160,132,10.25,68.20,8.7,6.72,9.9,12.0,38.588844
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,3165,530,55,189,4,4.971698,29,...,13,260,208,13.53,53.34,12.3,7.89,15.4,13.7,44.376204
5,1,"Dear @LOCATION1, I know having computers has a...",8,2569,473,36,205,4,4.431290,30,...,12,210,216,8.01,72.66,7.0,6.00,7.7,11.1,34.183440
